**TOPIC Modeling to find root cause of ticket**

In [1]:
# Mount the google drive
# Click on the link and sign in to google account. Get the authentication token and enter here
from google.colab import drive

drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
import pandas as pd

In [3]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
def pre_processing_data(dataframe,text_column_name):
    dataframe[text_column_name] = dataframe[text_column_name].apply(lambda x: ' '.join([item for item in x.split() if item not in stopwords.words('english')]))
    dataframe[text_column_name] = dataframe[text_column_name].str \
                                          .replace('From:', '').str \
                                          .replace('Resource:', '').str \
                                          .replace('Description:', '').str \
                                          .replace('Sent:', '').str \
                                          .replace('To:', '').str \
                                          .replace('Cc:', '').str \
                                          .replace('Metric Name:', '').str \
                                          .replace('-', '').str \
                                          .replace(':', '').str \
                                          .encode("ascii", "ignore").str \
                                          .decode("utf-8").str \
                                          .replace('\n', ' ').str.replace('\t','').str.replace('\r','').str \
                                          .replace('/\s\s+/g', ' ').str.strip()
    # Remove emails present in the data
    dataframe[text_column_name] = dataframe[text_column_name].apply(lambda x:' '.join([i for i in x.split() if '@' not in i])) 

    return dataframe 
        

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [27]:
data_t = pd.read_csv('gdrive/MyDrive/Train_AMS_NonAMS.csv')

In [28]:
data_t.head()

,"Please issue a ""High Priority"" ticket to Store Systems Application Services to recover the following missing transactions for Store # from //. This store was polled but we are still missing sales. Recover Missing Transactions:Store# Sales Date //Missing Transactions:Reg# - These are in electronic journal but not in ASReg# - These are in electronic journal but not in ASReg# - These are in electronic journal but not in ASReg# - These are in electronic journal but not in ASThis store has been reversed.Thanks for your time, Cathy PowellSr Accounting Clerk | Sales Auditt: .. | f:..Advance Auto Parts| Airport Road |Roanoke, VA Please take a few moments to provide feedback to help us serve you better! Click here to access the survey.",AMS
0,From: Tammy Hale <tammyhale@advance-auto.com> ...,AMS
1,From: Cathy Powell <CPOWELL@advance-auto.com> ...,AMS
2,missing transactions for Store # from //. This...,AMS
3,stating Missing sales data in Performance vs g...,AMS
4,From: Cathy Powell <CPOWELL@advance-auto.com> ...,AMS


In [33]:
 data_t.columns = ['Description','Category']

In [34]:
data_t['Description'] = data_t['Description'].astype(str)
data_t = pre_processing_data(data_t,'Description')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [35]:
data_t.head()

,Description,Category
0,"Tammy Hale Tuesday, August , PM Store Systems ...",AMS
1,"Cathy Powell Tuesday, August , AM Store System...",AMS
2,missing transactions Store # //. This store po...,AMS
3,stating Missing sales data Performance vs goal...,AMS
4,"Cathy Powell Monday, September , AM Store Syst...",AMS


In [36]:
data_t.shape

(2108, 2)

In [37]:
# Convert to list
data_list = data_t.Description.values.tolist()

In [38]:
data_list[0]

'Tammy Hale Tuesday, August , PM Store Systems Application Services POC Financial Sales Audit Team Missing Sales Store # //Good Morning Afternoon,Please poll Store # Sales Date //. We missing sales.This store reversed.Thank you,Tammy HaleSr. Accounting Administrator I Sales Auditt .. | f..Advance Auto Parts| Airport Road |Roanoke, VA Please take moments provide feedback help us serve better! Click access survey.'

In [39]:
import re
# Remove new line characters
data_list = [re.sub('\s+', ' ', sent) for sent in data_list]

# Remove distracting single quotes
data_list = [re.sub("\'", "", sent) for sent in data_list]

print(data_list[:1])

['Tammy Hale Tuesday, August , PM Store Systems Application Services POC Financial Sales Audit Team Missing Sales Store # //Good Morning Afternoon,Please poll Store # Sales Date //. We missing sales.This store reversed.Thank you,Tammy HaleSr. Accounting Administrator I Sales Auditt .. | f..Advance Auto Parts| Airport Road |Roanoke, VA Please take moments provide feedback help us serve better! Click access survey.']


<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<input>:3: DeprecationWarning: invalid escape sequence \s
<ipython-input-39-a527e5a12aea>:3: DeprecationWarning: invalid escape sequence \s
  data_list = [re.sub('\s+', ' ', sent) for sent in data_list]


In [40]:
import gensim
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data_list))

print(data_words[:1])

[['tammy', 'hale', 'tuesday', 'august', 'pm', 'store', 'systems', 'application', 'services', 'poc', 'financial', 'sales', 'audit', 'team', 'missing', 'sales', 'store', 'good', 'morning', 'afternoon', 'please', 'poll', 'store', 'sales', 'date', 'we', 'missing', 'sales', 'this', 'store', 'reversed', 'thank', 'you', 'tammy', 'halesr', 'accounting', 'administrator', 'sales', 'auditt', 'advance', 'auto', 'parts', 'airport', 'road', 'roanoke', 'va', 'please', 'take', 'moments', 'provide', 'feedback', 'help', 'us', 'serve', 'better', 'click', 'access', 'survey']]


In [41]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.7/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['tammy', 'hale', 'tuesday_august', 'pm', 'store', 'systems', 'application', 'services', 'poc', 'financial', 'sales', 'audit', 'team', 'missing', 'sales', 'store', 'good_morning', 'afternoon', 'please', 'poll', 'store', 'sales', 'date', 'we', 'missing', 'sales', 'this', 'store', 'reversed', 'thank_you', 'tammy', 'halesr', 'accounting', 'administrator', 'sales_auditt', 'advance', 'auto', 'parts', 'airport_road_roanoke_va', 'please', 'take_moments_provide_feedback', 'help_us_serve', 'better_click', 'access', 'survey']


In [57]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 'to', 'tm', 'called'])

In [60]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
import nltk; nltk.download('stopwords')

import spacy
def remove_stopwords(texts):

    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa


In [61]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])



# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(len(corpus))

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: Sele

2108


/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for entry_point in AVAILABLE_ENTRY_POINTS.get(self.entry_point_namespace, []):
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning: SelectableGroups dict interface is deprecated. Use sele

In [62]:
id2word[0]

'access'

In [63]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [64]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.136*"issue" + 0.116*"say" + 0.108*"state" + 0.087*"unable" + 0.060*"change" + 0.049*"user" + 0.036*"access" + 0.034*"able" + 0.033*"get" + 0.025*"can"'), (1, '0.159*"work" + 0.097*"printer" + 0.062*"pay" + 0.057*"reg" + 0.057*"feature" + 0.043*"connect" + 0.040*"label" + 0.033*"cable" + 0.032*"vital" + 0.028*"apple"'), (2, '0.185*"call" + 0.076*"need" + 0.065*"stop" + 0.057*"use" + 0.046*"miss" + 0.031*"monitor" + 0.026*"ping" + 0.024*"help" + 0.023*"replace" + 0.022*"today"'), (3, '0.215*"register" + 0.052*"apex" + 0.036*"lock" + 0.026*"serial" + 0.026*"log" + 0.025*"ring" + 0.023*"offline" + 0.023*"ipad" + 0.021*"tech" + 0.020*"come"'), (4, '0.222*"order" + 0.164*"customer" + 0.107*"online" + 0.033*"want" + 0.032*"close" + 0.022*"device" + 0.017*"place" + 0.015*"address" + 0.014*"cancel" + 0.012*"light"'), (5, '0.189*"error" + 0.158*"message" + 0.065*"update" + 0.025*"recovery" + 0.023*"response" + 0.023*"paste_pe" + 0.020*"statement" + 0.019*"hence" + 0.019*"multiple" + 0.01

In [47]:
!pip install pyLDAvis.gensim  

ERROR: Could not find a version that satisfies the requirement pyLDAvis.gensim (from versions: none)
ERROR: No matching distribution found for pyLDAvis.gensim


In [65]:
data_list = [re.sub("<", "", sent) for sent in data_list]
print(data_list[0])
print(data_bigram[0])

Tammy Hale Tuesday, August , PM Store Systems Application Services POC Financial Sales Audit Team Missing Sales Store # //Good Morning Afternoon,Please poll Store # Sales Date //. We missing sales.This store reversed.Thank you,Tammy HaleSr. Accounting Administrator I Sales Auditt .. | f..Advance Auto Parts| Airport Road |Roanoke, VA Please take moments provide feedback help us serve better! Click access survey.
['tammy', 'hale', 'tuesday_august', 'pm', 'store', 'systems', 'application', 'services', 'poc', 'financial', 'sales', 'audit', 'team', 'missing', 'sales', 'store', 'good_morning', 'afternoon', 'please', 'poll', 'store', 'sales', 'date', 'we', 'missing', 'sales', 'this', 'store', 'reversed', 'thank_you', 'tammy', 'halesr', 'accounting', 'administrator', 'sales', 'auditt', 'advance', 'auto', 'parts', 'airport_road', 'roanoke_va', 'please', 'take_moments', 'provide_feedback', 'help', 'us_serve', 'better_click', 'access', 'survey']


In [66]:
for i, row in enumerate(lda_model[corpus]):
  print (i, row)
  break

0 ([(0, 0.043890025), (2, 0.16100748), (7, 0.17068069), (11, 0.32769623), (14, 0.1952)], [(0, [0, 11]), (1, [2]), (2, [7]), (3, [7]), (4, [14]), (5, [11]), (6, [7, 14]), (7, [11]), (8, [2]), (9, [11]), (10, [2, 11]), (11, [2]), (12, [7, 14]), (13, [11]), (14, [11]), (15, [2]), (16, [11]), (17, [14]), (18, [14]), (19, [11]), (20, [14]), (21, [7]), (22, [7]), (23, [11])], [(0, [(0, 0.8312373), (11, 0.16876258)]), (1, [(2, 1.0)]), (2, [(7, 0.99999994)]), (3, [(7, 0.9999999)]), (4, [(14, 1.0)]), (5, [(11, 0.99999994)]), (6, [(7, 0.9883201), (14, 0.0116798505)]), (7, [(11, 0.99999994)]), (8, [(2, 1.0)]), (9, [(11, 1.0)]), (10, [(2, 0.86534524), (11, 0.13465475)]), (11, [(2, 2.0)]), (12, [(7, 0.91321933), (14, 0.08678069)]), (13, [(11, 1.0000001)]), (14, [(11, 1.0000001)]), (15, [(2, 0.99999994)]), (16, [(11, 5.0)]), (17, [(14, 1.0)]), (18, [(14, 3.9986012)]), (19, [(11, 1.0000001)]), (20, [(14, 0.9965174)]), (21, [(7, 1.0)]), (22, [(7, 1.0)]), (23, [(11, 1.0000001)])])


In [67]:
print(len(lda_model[corpus][0]))

3


In [68]:
# Plotting tools
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [69]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
7      0.282594 -0.070695       1        1  12.671235
0      0.323252  0.025064       2        1  10.608579
5     -0.027364 -0.028116       3        1   6.187974
14     0.132950  0.184404       4        1   5.542429
16     0.048487 -0.108971       5        1   5.427734
18    -0.040212  0.250853       6        1   5.202937
13    -0.080175  0.006053       7        1   4.769353
6     -0.014808 -0.063316       8        1   4.527045
9     -0.081641  0.004831       9        1   4.462143
2      0.003924 -0.046333      10        1   4.266327
3     -0.043736 -0.039137      11        1   4.018348
17    -0.023809 -0.112258      12        1   3.914784
11    -0.084065 -0.030005      13        1   3.885932
8     -0.086673  0.180017      14        1   3.844657
4     -0.059425 -0.016704      15        1   3.782495
12    -0.000025  0.105833      16        1   3.583313
19    -0.074082  0.008504      17        1   3.511450
1     -0.029424 -0.177370      18        1   3.424774
10    -0.060169 -0.040019      19        1   3.251006
15    -0.085598 -0.032636      20        1   3.117486, topic_info=              Term        Freq       Total Category  logprob  loglift
335           line  327.000000  327.000000  Default  30.0000  30.0000
527          phone  311.000000  311.000000  Default  29.0000  29.0000
58           error  317.000000  317.000000  Default  28.0000  28.0000
77           issue  548.000000  548.000000  Default  27.0000  27.0000
18           store  316.000000  316.000000  Default  26.0000  26.0000
...            ...         ...         ...      ...      ...      ...
1814          must    6.731551    7.724184  Topic20  -4.7952   3.3306
363        counter    6.489867    7.482502  Topic20  -4.8318   3.3258
2345  troubleshoot    6.434546    7.427179  Topic20  -4.8404   3.3247
1139           web    6.178207    7.170840  Topic20  -4.8810   3.3191
50            show   20.091541  122.547682  Topic20  -3.7017   1.6599

[692 rows x 6 columns], token_table=      Topic      Freq                Term
term                                     
1303      8  0.893157  abend_notification
116       2  0.990196                able
0         2  0.976995              access
0        13  0.009869              access
2619     19  0.810220     accessreference
...     ...       ...                 ...
2215     13  0.867370                  wt
2599      4  0.948602             yealink
1256     14  0.838474                year
73        2  0.958089           yesterday
422       9  0.945904                 yet

[681 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[8, 1, 6, 15, 17, 19, 14, 7, 10, 3, 4, 18, 12, 9, 5, 13, 20, 2, 11, 16])

In [ ]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_list):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_list)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(10)